In [123]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
%matplotlib inline

CWD = "/".join(os.getcwd().split("/")[:-2])

In [112]:
a = pd.read_csv(f"{CWD}/data/summary/lcs.csv")
b = pd.read_csv(f"{CWD}/data/summary/results.csv")
a["ID"] = a.apply(lambda x: f"{x.prefix} {x.ID}", axis=1)

In [113]:
df = a.merge(b, on="ID")
df

,ID,QCS,typ,cadence_min,mission,prefix,origin,h_mission,SpT,view_start,...,identifier,ED,e_ED,t0,e_t0,dur,e_dur,ampl,e_ampl,Prot
0,KIC 100004076,14,slc,1,kep,KIC,generic,Kepler,L1,1356.50,...,WISEP J190648.47+401106.8,10414,886,1358.52754,0.00072,0.49377,0.02388,0.79166,0.06277,8.88
1,EPIC 212035340,18,llc,30,ktwo,EPIC,generic,K2,M8,3437.00,...,MASS J08371832+2050349,1954494,483655,3437.83750,0.00029,0.05795,0.00631,298.07702,42.06769,4.63
2,TIC 237880881,1,slc,2,tess,TIC,generic,TESS,M5,1331.00,...,2MASS J01180670-6258591,6020,298,1331.82418,0.00194,0.77347,0.02975,0.46626,0.01576,8.43
3,TIC 44984200,10,slc,2,tess,TIC,custom,TESS,M6,1587.25,...,SCR J0838-5855,3809,135,1588.02740,0.00106,0.65066,0.03073,0.29153,0.00868,2.71
4,TIC 277539431,12,slc,2,tess,TIC,custom,TESS,M7,1641.00,...,WISEA J105515.71-735611.3,4495,394,1641.86831,0.00291,0.89720,0.08459,0.35356,0.03025,4.56


In [114]:
# mission ID, other ID, QCS, SpT, RA, Dec, vsini, Prot_d
#df["mission ID"] = df.apply(lambda x: f"{x.prefix} {x.ID}", axis=1)
df[r"$t_0$"] = df.apply(lambda x: f"{x.BJDoff + x.tstart:.3f}", axis=1)
df[r"ED"] = df.apply(lambda x: f"{x.ED}({x.e_ED})",axis=1)
df[r"$v\sin i$"] = df.apply(lambda x: f"{x.vsini_kms}({x.e_vsini_kms})",axis=1)

In [115]:
df1 = df.rename(index=str, columns={"identifier" : "other ID",
                                    "Gaia_G" : r"$G_\mathrm{Gaia}$",
                                    "Prot_d": r"$P_\mathrm{rot}$",
                                    "cadence_min":"cadence",
                                    "J":r"$J$"
                                   })

In [116]:
cols = ["ID", "QCS", "other ID", "RA", "Dec", r"$J$",
        "SpT", "cadence", r"$t_0$", r"$P_\mathrm{rot}$",
        r"$v\sin i$", "ED"]
units =  ["","","","","deg","deg","mag","min","BJD","d","km/s","s"]
df2 = df1[cols]
df3 = df2.append(dict(zip(cols,units)), ignore_index=True)
df3 = df3.sort_index(axis=0,ascending=False).reset_index()
df3 = df3[cols]
df3

,ID,QCS,other ID,RA,Dec,$J$,SpT,cadence,$t_0$,$P_\mathrm{rot}$,$v\sin i$,ED
0,,,,,deg,deg,mag,min,BJD,d,km/s,s
1,TIC 277539431,12,WISEA J105515.71-735611.3,163.815,-73.9364,10.63,M7,2,2458641.835,0.19,nan(nan),4495(394)
2,TIC 44984200,10,SCR J0838-5855,129.508,-58.9331,10.309,M6,2,2458588.030,0.113,nan(nan),3809(135)
3,TIC 237880881,1,2MASS J01180670-6258591,19.528,-62.9831,11.53,M5,2,2458331.700,0.35125,14.4(2.6),6020(298)
4,EPIC 212035340,18,MASS J08371832+2050349,129.326,20.843,15.9,M8,30,2458270.750,0.193,30.0(12.0),1954494(483655)
5,KIC 100004076,14,WISEP J190648.47+401106.8,286.7,40.1857,13.078,L1,1,2456191.550,0.37015,11.2(2.2),10414(886)


In [117]:
references = [("a","gizis2013",[]),
              ("b", "paudel2019",[]),
              ("c","kraus2014",["14.4(2.6)"]),
             ]

In [118]:
nc = 'c' * (df3.shape[1]-2) #number of middle columns
footnote_pref= "\multicolumn{" + str(df3.shape[1]-2) + "}{l}" 
footnote_suf = "\n"
fs = []
for alpha, ref, repl in references:
    f1 = footnote_pref + "{$^" + alpha + "$}{\citet\{" + ref + "}}" + footnote_suf
    fs.append(f1)

stri = df3.to_latex(index=False,escape=False, column_format=f"l{nc}r")
stri = stri.replace("\midrule","\hline")

for alpha, ref, repl in references:
    if len(repl) > 0:
        for r in repl:
            stri = stri.replace(r,r + r"$^" + alpha + "$")

repl = " ".join(fs)
stri = stri.replace("\\bottomrule","\hline\n" + repl)
print(stri)

\begin{tabular}{lccccccccccr}
\toprule
             ID & QCS &                   other ID &       RA &      Dec &     $J$ &  SpT & cadence &        $t_0$ & $P_\mathrm{rot}$ &   $v\sin i$ &               ED \\
\hline
                &     &                            &          &      deg &     deg &  mag &     min &          BJD &                d &        km/s &                s \\
  TIC 277539431 &  12 &  WISEA J105515.71-735611.3 &  163.815 & -73.9364 &   10.63 &   M7 &       2 &  2458641.835 &             0.19 &    nan(nan) &        4495(394) \\
   TIC 44984200 &  10 &             SCR J0838-5855 &  129.508 & -58.9331 &  10.309 &   M6 &       2 &  2458588.030 &            0.113 &    nan(nan) &        3809(135) \\
  TIC 237880881 &   1 &    2MASS J01180670-6258591 &   19.528 & -62.9831 &   11.53 &   M5 &       2 &  2458331.700 &          0.35125 &   14.4(2.6)$^c$ &        6020(298) \\
 EPIC 212035340 &  18 &    MASS J08371832+2050349  &  129.326 &   20.843 &    15.9 &   M8 &      30 

In [119]:
## Add references

In [120]:
with open(f"{CWD}/paper/draft/tables/lcs.tex", "w") as f:
    f.write(stri)